# simple nn model fitting

Notebook will run a very simple nn on the training points

In [1]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
# import librosa
# import librosa.display
# import sklearn as skl
# import sklearn.utils
# import IPython.display as ipd
# from sklearn.model_selection import train_test_split
# import ast

from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F

In [65]:
#define network class
#used some help from https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

class Net(nn.Module):
    def __init__(self, input_size, output_size = 1):
        super(Net, self).__init__()
        #linear layers to keep decrease the number of nodes
        self.layer1 = nn.Linear(input_size, int(input_size/2))
        self.layer2 = nn.Linear(int(input_size/2), int(input_size/4))
        self.layer3 = nn.Linear(int(input_size/4), int(input_size/8))
        self.layer4 = nn.Linear(int(input_size/8), int(input_size/16))
        self.last = nn.Linear(int(input_size/16), output_size)

    # x represents our data
    def forward(self, x):
        #these are a bunch of linear layers with relu activation
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = self.last(x)
        return x

        

In [66]:
#import training data
train_data = torch.tensor(np.load('data/train_data.npy').T).float()[:1000]
labels = torch.tensor(np.load('data/train_labels.npy')).float()
n_dim, n_training_points = train_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_training_points )

#get testing data
test_data = torch.tensor(np.load('data/test_data.npy').T).float()[:1000]
test_labels = torch.tensor(np.load('data/test_labels.npy')).float()
_, n_testing_points = test_data.shape
print("n_dim = ", n_dim)
print('n_training_points = ', n_testing_points )
print(test_data)


#one hot encode the genre labels
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(torch.cat((labels, test_labels)).unsqueeze(1)) #takes a 2d array as input and labels is 1d; need labels for all training and testing points
print(len(enc.categories_))
labels = enc.transform(labels.unsqueeze(1)).toarray()
labels = torch.tensor(labels).float()
print(labels.shape)
_, n_categories = labels.shape #number of categories is out output size for our network


#encode testing labels
test_labels = enc.transform(test_labels.unsqueeze(1)).toarray()
test_labels = torch.tensor(test_labels).float()
print(test_labels.shape)


n_dim =  1000
n_training_points =  50
n_dim =  1000
n_training_points =  25
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0077, -0.1812, -0.1608,  ...,  0.2518,  0.1378,  0.1512],
        [-0.0131, -0.1836, -0.1614,  ...,  0.2579,  0.1327,  0.1578],
        [-0.0178, -0.1850, -0.1577,  ...,  0.2488,  0.1278,  0.1573]])
1
torch.Size([50, 18])
torch.Size([25, 18])


In [67]:
#define the model, optimizer and loss function
model = Net(n_dim, n_categories)
print(model)
optimizer = torch.optim.Adam(model.parameters()) #adam is a "better" version of stochastic gradient decent
criterion = nn.MSELoss()

Net(
  (layer1): Linear(in_features=1000, out_features=500, bias=True)
  (layer2): Linear(in_features=500, out_features=250, bias=True)
  (layer3): Linear(in_features=250, out_features=125, bias=True)
  (layer4): Linear(in_features=125, out_features=62, bias=True)
  (last): Linear(in_features=62, out_features=18, bias=True)
)


In [68]:
max_epochs = 200
max_test_loss = 1e10
for epoch in range(max_epochs):
    optimizer.zero_grad()   # zero the gradient buffers
    output = model(train_data.T)#put through model
    loss = criterion(output, labels)#find loss
    test_output = model(test_data.T) 
    test_loss = criterion(test_output, test_labels)
    if(test_loss< max_test_loss):
        max_test_loss = test_loss
        torch.save(model, "model.pt")
        print("model saved at epoch ", epoch)
    loss.backward() #do the back propogation
    optimizer.step()    # Does the update
    
    print("loss = ", loss, ": test_loss = ", test_loss)
    

model saved at epoch  0
loss =  tensor(0.0648, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0573, grad_fn=<MseLossBackward>)
model saved at epoch  1
loss =  tensor(0.0625, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0564, grad_fn=<MseLossBackward>)
model saved at epoch  2
loss =  tensor(0.0604, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0557, grad_fn=<MseLossBackward>)
model saved at epoch  3
loss =  tensor(0.0583, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0550, grad_fn=<MseLossBackward>)
model saved at epoch  4
loss =  tensor(0.0562, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0545, grad_fn=<MseLossBackward>)
model saved at epoch  5
loss =  tensor(0.0540, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0542, grad_fn=<MseLossBackward>)
model saved at epoch  6
loss =  tensor(0.0516, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0542, grad_fn=<MseLossBackward>)
loss =  tensor(0.0493, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.0543, gra

loss =  tensor(0.0128, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1137, grad_fn=<MseLossBackward>)
loss =  tensor(0.0125, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1135, grad_fn=<MseLossBackward>)
loss =  tensor(0.0123, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1137, grad_fn=<MseLossBackward>)
loss =  tensor(0.0120, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1143, grad_fn=<MseLossBackward>)
loss =  tensor(0.0118, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1165, grad_fn=<MseLossBackward>)
loss =  tensor(0.0116, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1170, grad_fn=<MseLossBackward>)
loss =  tensor(0.0114, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1170, grad_fn=<MseLossBackward>)
loss =  tensor(0.0111, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1182, grad_fn=<MseLossBackward>)
loss =  tensor(0.0110, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1196, grad_fn=<MseLossBackward>)
loss =  tensor(0.0107, grad_fn=<MseLo

loss =  tensor(0.0046, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1971, grad_fn=<MseLossBackward>)
loss =  tensor(0.0045, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2014, grad_fn=<MseLossBackward>)
loss =  tensor(0.0044, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2000, grad_fn=<MseLossBackward>)
loss =  tensor(0.0044, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2001, grad_fn=<MseLossBackward>)
loss =  tensor(0.0044, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2030, grad_fn=<MseLossBackward>)
loss =  tensor(0.0043, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1970, grad_fn=<MseLossBackward>)
loss =  tensor(0.0042, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.2008, grad_fn=<MseLossBackward>)
loss =  tensor(0.0042, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1984, grad_fn=<MseLossBackward>)
loss =  tensor(0.0041, grad_fn=<MseLossBackward>) : test_loss =  tensor(0.1989, grad_fn=<MseLossBackward>)
loss =  tensor(0.0041, grad_fn=<MseLo

In [69]:
model = torch.load('model.pt')

In [70]:
#find the classification error of the training data 
count = 0

output = model(train_data.T)
loss = criterion(output, labels)
print(" loss = ", loss)
#count number predicted correctly
for i in range(len(output)):
    idx_output = torch.argmax(output[i])
    idx_truth = torch.argmax(labels[i])
    if idx_output == idx_truth:
        count += 1
    else:
        pass #do nothing if not equal
print("percent predicted correctly is ", 100*count/len(output), '%')
            

 loss =  tensor(0.0516, grad_fn=<MseLossBackward>)
percent predicted correctly is  16.0 %


In [71]:
#find the classification of the test data
count = 0
test_output = model(test_data.T)
test_loss = criterion(test_output, test_labels)
print("test loss = ", test_loss)
#count number predicted correctly
for i in range(len(test_output)):
    idx_output = torch.argmax(test_output[i])
    idx_truth = torch.argmax(test_labels[i])
#     print("idx_output = ", idx_output, ": idx_truth = ", idx_truth)
    if idx_output == idx_truth:
        count += 1
    else:
        pass #do nothing if not equal
print("percent predicted correctly is ", 100*count/len(output), '%')

test loss =  tensor(0.0542, grad_fn=<MseLossBackward>)
percent predicted correctly is  2.0 %
